In [ ]:
import numpy as np
from scipy import sparse
import networkx as nx
from epynet import Network
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns

In [ ]:
wds_name = 'anytown'

In [ ]:
if wds_name == 'anytown':
    wds = Network('../water_networks/anytown.inp')
elif wds_name == 'ctown':
    wds = Network('../water_networks/ctown.inp')
elif wds_name == 'richmond':
    wds = Network('../water_networks/richmond.inp')

### NetworkX hálózat felépítése
Az éleket az ellenállástényezővel súlyozzuk. A gráf konvolúciós technikák  a **Spektral** csomagban csak bináris szomszédsági mátrixokat kezelnek, a súlyozást az élek attribútumaként kell kezelni.

In [ ]:
G = nx.Graph()

max_weight = 0
for pipe in wds.pipes:
    #weight = (pipe.diameter/1000)**5 / (pipe.roughness * pipe.length)
    weight = (pipe.diameter*0.03937)**4.8655 / (0.2083 * (100/pipe.roughness)**1.852 * pipe.length)
    G.add_edge(pipe.from_node.index, pipe.to_node.index, weight=weight)
    if weight > max_weight:
        max_weight = weight
for pump in wds.pumps:
    G.add_edge(pump.from_node.index, pump.to_node.index, weight=1.)
for valve in wds.pumps:
    G.add_edge(valve.from_node.index, valve.to_node.index, weight=1.)

for (_,_,d) in G.edges(data=True):
    d['weight'] /= max_weight

In [ ]:
if nx.is_connected(G.to_undirected()):
    print(nx.algorithms.distance_measures.diameter(G.to_undirected()))
else:
    for comp in list(nx.connected_components(G)):
        print(nx.algorithms.distance_measures.diameter(G.subgraph(comp)))

### Súlyozott szomszédsági mátrix

In [ ]:
A_tilde = np.array(nx.adjacency_matrix(G).toarray())
A_tilde += np.eye(np.shape(A_tilde)[0])
D_tilde = np.diag(np.sum(A_tilde, axis=0)**-.5)
Laplacian_weighted = np.dot(np.dot(D_tilde, A_tilde), D_tilde)

### Bináris szomszédsági mátrix

In [ ]:
A_tilde = np.array(nx.adjacency_matrix(G).toarray())
A_tilde += np.eye(np.shape(A_tilde)[0])
A_tilde[np.nonzero(A_tilde)] = 1
D_tilde = np.diag(np.sum(A_tilde, axis=0)**-.5)
Laplacian_binary = np.dot(np.dot(D_tilde, A_tilde), D_tilde)

### Gráf Laplace-mátrixa súlyozott és bináris szomszédsággal

In [ ]:
figsize = (8,6)
axlabel = "Node number"
barlabel = "Value of Laplacian"
fig, ax = plt.subplots(figsize=figsize)
hmap = sns.heatmap(Laplacian_weighted,
                  norm = LogNorm(vmin=Laplacian_weighted[Laplacian_weighted.nonzero()].min(),
                               vmax=Laplacian_weighted.max()),
                  cmap = 'viridis_r',
                  cbar_kws = {'label': barlabel},
                  #linewidths=.01, linecolor='grey',
                  square=True,
                  ax = ax
                 )

#fig.set_title('Laplacian of graph with weighted connections')
hmap.set_xlabel(axlabel)
hmap.set_ylabel(axlabel)
#fig.axis('off')

fn = 'Laplacian-weighted-' + wds_name + '.pdf'
#fig.savefig(fn, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=figsize, edgecolor='k')

hmap = sns.heatmap(Laplacian_binary, 
            norm = LogNorm(vmin=Laplacian_binary[Laplacian_binary.nonzero()].min(),
                         vmax=Laplacian_binary.max()),
                  cmap='viridis_r',
                  cbar_kws = {'label': barlabel},
                  #linewidths=.000001, linecolor='grey',
                  square = True,
                  ax = ax
                 )

#fig.set_title('Laplacian of graph with binary connections')
#fig.axis('off')

hmap.set_xlabel(axlabel)
hmap.set_ylabel(axlabel)

fn = 'Laplacian-binary-' + wds_name + '.pdf'
#fig.savefig(fn, bbox_inches='tight')

### Csomópontok saját fontossága a szomszédaihoz képest

In [ ]:
center_weighted = Laplacian_weighted.diagonal()
radius_weighted = np.sum(Laplacian_weighted, axis=0)-center_weighted
ratio_weighted = radius_weighted/center_weighted

center_binary = Laplacian_binary.diagonal()
radius_binary = np.sum(Laplacian_binary, axis=0)-center_binary
ratio_binary = radius_binary/center_binary

In [ ]:
xlab = 'Node number'
ylab = 'Self-importance compared to neighbours'
fig, ax = plt.subplots(figsize=(6, 5)) # set the size that you'd like (width, height)
plt.bar(np.arange(len(ratio_weighted)), 1/ratio_weighted, log=True)
plt.xlabel(xlab)
plt.ylabel(ylab)

fn = 'Self-importance_weighted-' + wds_name + '.pdf'
#fig.savefig(fn, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5)) # set the size that you'd like (width, height)
plt.bar(np.arange(len(ratio_binary)), 1/ratio_binary, log=True)
plt.xlabel(xlab)
plt.ylabel(ylab)

fn = 'Self-importance_binary-' + wds_name + '.pdf'
#fig.savefig(fn, bbox_inches='tight')